In [7]:
import os
import sys
import csv
import base64
import time
from pathlib import Path

import numpy as np

import json
import cv2 as cv
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
from tqdm import tqdm

tiny,fast = True,False

TINY_IMG_NUM = 512
FAST_IMG_NUM = 5000

csv.field_size_limit(sys.maxsize)
FIELDNAMES = ["img_id", "img_h", "img_w", "objects_id", "objects_conf",
              "attrs_id", "attrs_conf", "num_boxes", "boxes", "features"]
SPLIT2NAME = {
    'train': 'train2014',
    'valid': 'val2014',
    'minival': 'val2014',
    'nominival': 'val2014',
    'test': 'test2015',
}


## Loading tsv

In [8]:
split = 'minival'

home = str(Path.home())
MSCOCO_IMGFEAT_ROOT = home + '/data/mscoco_imgfeat/'

In [27]:
fname = MSCOCO_IMGFEAT_ROOT+ '%s_obj36.tsv' % SPLIT2NAME[split]

In [28]:
fname

'/home/u37216/data/mscoco_imgfeat/val2014_obj36.tsv'

In [29]:
import pandas as pd

In [49]:
data = []
with open(fname) as f:
        reader = csv.DictReader(f, FIELDNAMES, delimiter="\t")
        for i, item in tqdm(enumerate(reader)):

            for key in ['img_h', 'img_w', 'num_boxes']:
                item[key] = int(item[key])
            
            boxes = item['num_boxes']
            decode_config = [
                ('objects_id', (boxes, ), np.int64),
                ('objects_conf', (boxes, ), np.float32),
                ('attrs_id', (boxes, ), np.int64),
                ('attrs_conf', (boxes, ), np.float32),
                ('boxes', (boxes, 4), np.float32),
                ('features', (boxes, -1), np.float32),
            ]
            for key, shape, dtype in decode_config:
                item[key] = np.frombuffer(base64.b64decode(item[key]), dtype=dtype)
                item[key] = item[key].reshape(shape)
                item[key].setflags(write=False)

            data.append(item)
            break


0it [00:00, ?it/s]


In [54]:
data[0].keys()

odict_keys(['img_id', 'img_h', 'img_w', 'objects_id', 'objects_conf', 'attrs_id', 'attrs_conf', 'num_boxes', 'boxes', 'features'])

In [55]:
data[0]['img_id']

'COCO_val2014_000000568281'

In [56]:
data[0]['img_h']

480

In [57]:
data[0]['img_w']

640

In [58]:
data[0]['objects_id']

array([ 43, 248, 200, 248, 865, 106, 240, 545, 847, 125, 231, 231, 106,
       945, 314, 200, 106, 545, 847, 945, 765, 240,  46, 610, 498, 262,
       106, 248, 242, 545, 106, 200, 610, 262, 248, 865])

In [59]:
data[0]['objects_conf']

array([0.99627316, 0.8498162 , 0.8100942 , 0.7666107 , 0.7041664 ,
       0.6968603 , 0.667069  , 0.64445174, 0.5755285 , 0.51019496,
       0.47075778, 0.4523234 , 0.39749786, 0.37325257, 0.3441021 ,
       0.3397924 , 0.32524255, 0.28065214, 0.2781245 , 0.2958179 ,
       0.25644326, 0.25172013, 0.23784612, 0.23489572, 0.2179063 ,
       0.21028809, 0.3364066 , 0.2073744 , 0.1892904 , 0.18810439,
       0.18295196, 0.367575  , 0.2264468 , 0.19824098, 0.23045404,
       0.13941209], dtype=float32)

In [65]:
data[0]['attrs_id']

array([261,   7,  23,   7,  23,   7,   6, 193,  24,  24,   6,   6,  24,
         7,   7,  23,  24,   7,  24,   7,  11,   6,   7,  24,  24, 319,
        24,   7,   7,  11,  23,  23,  24,   7,   7,  11])

In [62]:
data[0]['attrs_conf']

array([0.14530829, 0.41690323, 0.09729921, 0.31454325, 0.13444857,
       0.19824159, 0.49741054, 0.22050609, 0.06622802, 0.17859027,
       0.55552775, 0.16961418, 0.1577517 , 0.16778152, 0.24167347,
       0.11032619, 0.26237318, 0.29553568, 0.15619402, 0.14970426,
       0.11055475, 0.30072224, 0.29968244, 0.25049028, 0.2235659 ,
       0.01341583, 0.09030344, 0.08166693, 0.11218552, 0.16439971,
       0.14359647, 0.11391702, 0.23610055, 0.0440248 , 0.10836915,
       0.11838263], dtype=float32)

In [63]:
data[0]['num_boxes']

36

In [66]:
data[0]['boxes'].shape

(36, 4)

In [67]:
data[0]['features'].shape

(36, 2048)

In [43]:
t = pd.read_csv(fname, delimiter="\t",chunksize=1)